In [ ]:
import galsim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
j0 = galsim.bessel.j0
int1d = galsim.integ.int1d

In [ ]:
class Fphi:
    def __init__(self, r0, L0):
        self.r0 = r0
        self.L0 = L0
    def __call__(self, kappa):
        return 0.033/0.423*self.r0**(-5./3)*(kappa**2+self.L0**(-2))**(-11./6)

In [ ]:
class Dphi:
    def __init__(self, r0, L0, kappa_min=0.0):
        self.r0 = r0
        self.L0 = L0
        self.kappa_min = kappa_min
        self.Fphi_ = Fphi(r0, L0)
    def _integrand(self, rho):
        return lambda kappa: self.Fphi_(kappa)*(1-j0(rho*kappa))*kappa
    def __call__(self, rho):
        return 8*np.pi**2*int1d(self._integrand(rho), self.kappa_min, np.inf)

In [ ]:
class CanonicalDphi:
    def __init__(self, beta_min=0.0):
        self.beta_min = beta_min
    def _integrand(self, nu):
        return lambda beta: (1.0+beta*beta)**(-11./6)*(1.0-j0(nu*beta))*beta
    def __call__(self, nu):
        return int1d(self._integrand(nu), self.beta_min, np.inf)

In [ ]:
class ScaledDphi:
    def __init__(self, r0, L0=np.inf, kappa_min=0.0):
        self.r0 = r0
        if L0 > 1e100:
            L0 = 1e100
        self.L0 = L0
        self.kappa_min = kappa_min
        self.beta_min = self.L0*kappa_min
    magic = 0.033/0.423*8*np.pi**2
    def __call__(self, rho):
        return self.magic*CanonicalDphi(beta_min=self.beta_min)(rho/self.L0) * (self.r0/self.L0)**(-5./3)

In [ ]:
class MTF:
    def __init__(self, lam, r0, L0=np.inf, kappa_min=0.0):
        self.lam = lam
        self.r0 = r0
        self.L0 = L0
        self.kappa_min = kappa_min
        self.dphi = ScaledDphi(r0, L0, kappa_min)
    def __call__(self, k):
        # k in inverse radians
        rho = self.lam*k/(2*np.pi)
        return np.exp(-0.5*(self.dphi(rho)))

In [ ]:
class ScaledMTF:
    def __init__(self, lam, r0, L0=np.inf, kappa_min=0.0):
        self.lam = lam
        self.r0 = r0
        self.L0 = L0
        self.kappa_min = kappa_min
        self.dphi = Dphi(r0, L0, kappa_min)
    def __call__(self, k):
        # k in inverse radians
        rho = self.lam*k/(2*np.pi)
        return np.exp(-0.5*(self.dphi(rho)))

In [ ]:
# Are Dphi and ScaledDphi the same?
lam = 500e-9
r0 = 0.2
L0 = 1e4
kappa_min = 1.0
dphi = Dphi(r0, L0, kappa_min)
sdphi = ScaledDphi(r0, L0, kappa_min)

rhos = [0.01, 0.1, 0.2, 0.3, 0.4, 1.0, 10.0, 20.0]
for rho in rhos:
    print(dphi(rho), sdphi(rho), dphi(rho)/sdphi(rho))

In [ ]:
# Is MTF the same as what comes out of galsim.Kolmogorov?
lam = 500e-9
r0 = 0.2
L0 = 1e5
kappa_min = 10

kolm = galsim.Kolmogorov(r0=r0, lam=lam*1e9)
print(kolm.stepK(), kolm.maxK())
kmtf = lambda k: kolm.kValue(k, 0.0).real
mtf = MTF(lam, r0, L0, kappa_min)
smtf = ScaledMTF(lam, r0, L0, kappa_min)

ks = [0.0, 1.0, 10.0, 25.0]
print("MTF vs Kolmogorov")
for k in ks:
    print(k, mtf(k*galsim.radians/galsim.arcsec), kmtf(k), mtf(k*galsim.radians/galsim.arcsec)/kmtf(k))
print()
print("ScaledMTF vs Kolmogorov")
for k in ks:
    print(k, smtf(k*galsim.radians/galsim.arcsec), kmtf(k), smtf(k*galsim.radians/galsim.arcsec)/kmtf(k))
print()
print("MTF vs ScaledMTF")
for k in ks:
    print(k, mtf(k*galsim.radians/galsim.arcsec), smtf(k*galsim.radians/galsim.arcsec), mtf(k*galsim.radians/galsim.arcsec)/smtf(k*galsim.radians/galsim.arcsec))

# Note conclusions:  ScaledMTF works pretty well even when L0 = 1e20.  Regular MTF can't handle this though.

In [ ]:
ks = np.linspace(0.0, 250.0)
plt.figure()
plt.plot(ks, [smtf(k*galsim.radians/galsim.arcsec) for k in ks], label="MTF")
plt.plot(ks, [kmtf(k) for k in ks], label="Kolmogorov")
plt.xlabel("k (arcsec^-1)")
plt.ylabel("MTF")
plt.legend()
plt.show()

In [ ]:
# Not going to even bother wrapping the regular MTF here.  It's straight to ScaledMTF.
class PSF:
    def __init__(self, lam, r0, L0=np.inf, kappa_min=0.0):
        self.lam = lam
        self.r0 = r0
        self.L0 = L0
        self.kappa_min = kappa_min
        self.mtf = ScaledMTF(lam, r0, L0, kappa_min)
    def __call__(self, alpha):
        return int1d(lambda k: self.mtf(k)*j0(alpha*k)*k, 0, 500*206265)

In [ ]:
lam = 500e-9
r0 = 0.2
L0 = 1e2
kappa_min = 3
psf = PSF(lam, r0, L0, kappa_min)
kolm = galsim.Kolmogorov(r0=r0, lam=lam*1e9)

a0 = psf(0)
ak0 = kolm.xValue(0,0)

alphas = np.linspace(0.0, 1.5, 20)
psfvk = [psf(a*galsim.arcsec/galsim.radians)/a0 for a in alphas]
psfk = [kolm.xValue(a,0)/ak0 for a in alphas]

plt.plot(alphas, psfvk, label='vK')
plt.plot(alphas, psfk, label='Kolm')
plt.legend()
plt.xlabel("arcsec")
plt.ylabel("PSF")
plt.show()

plt.plot(alphas, psfvk, label='vK')
plt.plot(alphas, psfk, label='Kolm')
plt.legend()
plt.xlabel("arcsec")
plt.ylabel("PSF")
plt.yscale('log')
plt.show()

In [ ]:
lam = 500e-9
r0 = 0.2
L0 = 1e2
kappa_min = 3
psf = PSF(lam, r0, L0, kappa_min)
kolm = galsim.Kolmogorov(r0=r0, lam=lam*1e9)

a0 = psf(0)
ak0 = kolm.xValue(0,0)

alphas = np.linspace(0.0, 1.5, 20)
psfvk = [psf(a*galsim.arcsec/galsim.radians)/a0 for a in alphas]
psfk = [kolm.xValue(a,0)/ak0 for a in alphas]

plt.plot(alphas, psfvk, label='vK')
plt.plot(alphas, psfk, label='Kolm')
plt.legend()
plt.xlabel("arcsec")
plt.ylabel("PSF")
plt.show()

plt.plot(alphas, psfvk, label='vK')
plt.plot(alphas, psfk, label='Kolm')
plt.legend()
plt.xlabel("arcsec")
plt.ylabel("PSF")
plt.yscale('log')
plt.show()